# Imports

In [1]:
using QuadGK
using Interpolations
using OrdinaryDiffEq

In [2]:
include("/home/jaimerz/PhD/LimberJack.jl/src/core.jl")

angularCℓ (generic function with 1 method)

In [3]:
cosmoPar = CosmoPar(0.27, 0.049, 0.67, 0.96, 0.81)
cosmo = Cosmology(cosmoPar);

┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#51 at convenience-constructors.jl:7 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:7
┌ Warning: extrapolate(itp, Linear()) is deprecated, use extrapolate(itp, Line()) instead
│   caller = #LinearInterpolation#52 at convenience-constructors.jl:9 [inlined]
└ @ Core /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9


# Data

In [4]:
z_arr = [0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,  1]
b_arr = [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]
nz = [[0.0, 0.1, 0.2, 0.30000000000000004,
       0.4, 0.5, 0.6000000000000001, 0.7000000000000001,
       0.8, 0.9, 1.0],
      [0.0, 0.09549150281252627, 0.3454915028125263,
       0.6545084971874737, 0.9045084971874736, 1.0,
       0.9045084971874736, 0.6545084971874737, 0.34549150281252644,
       0.09549150281252634, 0]]
clustering_t = [0.00000000e+00, 4.48526950e-05, 1.71324904e-04, 3.43709458e-04,
        5.04105075e-04, 5.92283813e-04, 5.69713631e-04, 4.38454138e-04,
        2.46077204e-04, 7.22670075e-05, 1.03428840e-10]
lensing_t = [0.00000000e+00, 8.46720571e-06, 1.23677281e-05, 1.23488595e-05,
        9.80438814e-06, 6.34013396e-06, 3.25773350e-06, 1.22774325e-06,
        2.76337744e-07, 1.90522187e-08, 0.00000000e+00]
CMBlensing_t = [0.00000000e+00, 1.11161519e-05, 2.29070311e-05, 3.51531866e-05,
        4.76791214e-05, 6.03482340e-05, 7.30566167e-05, 8.57272238e-05,
        9.83037415e-05, 1.10746285e-04, 1.23027399e-04];
ell_arr = [ 31.5,  74. , 120. , 169.5, 223.5, 281.5, 344.];
lensing_lensing_cl_t = [6.96479316e-09, 2.62291354e-09, 1.39551291e-09, 8.99582785e-10,
       6.43044113e-10, 4.92082002e-10, 3.92524368e-10]
clust_lensing_cl_t = [2.66277949e-07, 1.14529794e-07, 6.13494474e-08, 3.84973684e-08,
       2.65390829e-08, 1.96776245e-08, 1.53408900e-08]
clust_clust_cl_t = [2.22573773e-05, 1.08124919e-05, 6.17595003e-06, 3.83844129e-06,
       2.61700894e-06, 1.88608858e-06, 1.43344030e-06]
CMBk_CMBk_cl_t = [2.13619136e-07, 1.84110942e-07, 1.39760222e-07, 1.05266694e-07,
       8.05684880e-08, 6.31640681e-08, 5.03579347e-08]
CMBlensing_clust_cl_t = [1.31477844e-06, 6.48418679e-07, 3.79843276e-07, 2.36573072e-07,
       1.62207584e-07, 1.16943272e-07, 8.86539366e-08]


7-element Array{Float64,1}:
 1.31477844e-6
 6.48418679e-7
 3.79843276e-7
 2.36573072e-7
 1.62207584e-7
 1.16943272e-7
 8.86539366e-8

# Compare Tracers

In [5]:
pz = get_pz(nz);

In [6]:
NC = NumberCountsTracer(nz[1], nz[2], 2)

MethodError: [91mMethodError: no method matching LinearInterpolation(::Interpolations.FilledExtrapolation{Float64,1,Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear{Throw{OnGrid}}},Tuple{Array{Float64,1}}},Gridded{Linear{Throw{OnGrid}}},Int64}, ::Int64; extrapolation_bc=0)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  LinearInterpolation(::AbstractArray{T,1} where T, [91m::AbstractArray{T,1} where T[39m; extrapolation_bc) at /home/jaimerz/.julia/packages/Interpolations/3gTQB/src/convenience-constructors.jl:9[39m

In [7]:
qg = clustering_kernel(cosmo, 2, nz)
clustering_jl = [qg(z, 1) for z in z_arr]

11-element Array{Array{Float64,1},1}:
 [0.0]
 [8.909770650135775e-5]
 [0.0003378459382687228]
 [0.0006730393946546217]
 [0.0009806530052339985]
 [0.0011452530594967406]
 [0.0010956040239402866]
 [0.0008390634300406726]
 [0.00046887060833419056]
 [0.0001371674456204168]
 [0.0]

In [8]:
NC_t = [NC(z) for z in z_arr]

UndefVarError: [91mUndefVarError: NC not defined[39m

In [9]:
clustering_jl./clustering_t

11-element Array{Array{Float64,1},1}:
 [NaN]
 [1.9864515722267693]
 [1.971960470316229]
 [1.9581637309923012]
 [1.9453345222402263]
 [1.9336220817784542]
 [1.9230784807047854]
 [1.9136857365927575]
 [1.9053801031248332]
 [1.8980645576118094]
 [0.0]

In [10]:
lensing_jl = [lensing_kernel(cosmo, z, nz) for z in z_arr]

11-element Array{Float64,1}:
 0.0
 7.193789890103526e-6
 1.0586957127444778e-5
 1.0668050823628814e-5
 8.569027857054874e-6
 5.631462924664495e-6
 2.96392884921399e-6
 1.167696126260265e-6
 2.9477092534508695e-7
 3.224829887406658e-8
 0.0

In [11]:
lensing_jl./lensing_t

11-element Array{Float64,1}:
 NaN
   0.8496061317616819
   0.8560147055177239
   0.8638895619169377
   0.8739992475506865
   0.888224595914452
   0.9098131720148348
   0.9510914649787445
   1.0667052610268357
   1.69262695236994
 NaN

In [12]:
qCMBL = CMBlensing_kernel(cosmo)
CMBlensing_jl = [qCMBL(z, 1) for z in z_arr]

MethodError: [91mMethodError: no method matching CMBlensing_kernel(::Cosmology)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  CMBlensing_kernel(::Cosmology, [91m::Any[39m, [91m::Any[39m) at /home/jaimerz/PhD/LimberJack.jl/src/core.jl:182[39m

In [13]:
CMBlensing_jl./CMBlensing_t

UndefVarError: [91mUndefVarError: CMBlensing_jl not defined[39m

In [14]:
CMBlensing_jl./CMBlensing_t

UndefVarError: [91mUndefVarError: CMBlensing_jl not defined[39m

# Compare Cls

In [15]:
qgamma = shear_kernel(cosmo, nz)
qk = convergence_kernel(cosmo, nz)
qCMBk = CMBk_kernel(cosmo, 1100)
qg = clustering_kernel(cosmo, 2, nz)

qg (generic function with 1 method)

In [16]:
CMBlensing_kernel(cosmo, 3, 1100)

0.0005508201348330815

In [17]:
clust_clust_cl_jl = Cl(cosmo, ell_arr, qg, qg)

7-element Array{Float64,1}:
 2.564529775205927e-5
 1.2045642294945372e-5
 6.232113514485507e-6
 3.555520295979774e-6
 2.1668026880569987e-6
 1.3956315608269605e-6
 9.358623476957292e-7

In [ ]:
conv_conv_cl_jl = Cl(cosmo, ell_arr, qk, qk)

In [ ]:
shear_shear_cl_jl = Cl(cosmo, ell_arr, qgamma, qgamma)

In [ ]:
CMBk_CMBk_cl_jl = Cl(cosmo, ell_arr, qCMBk, qCMBk)

In [ ]:
clust_clust_cl_jl./clust_clust_cl_t

In [ ]:
conv_conv_cl_jl./lensing_lensing_cl_t

In [ ]:
shear_shear_cl_jl./lensing_lensing_cl_t

In [ ]:
CMBk_CMBk_cl_jl./CMBk_CMBk_cl_t